# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import pandas as pd
from sqlalchemy import create_engine

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
    

[nltk_data] Downloading package punkt to /Users/nhanvo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nhanvo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nhanvo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table("message", engine)

X = df["message"]
Y = df.iloc[:, 4:]
print(Y.astype(int).max())

related                   1
request                   1
offer                     1
aid_related               1
medical_help              1
medical_products          1
search_and_rescue         1
security                  1
military                  1
child_alone               0
water                     1
food                      1
shelter                   1
clothing                  1
money                     1
missing_people            1
refugees                  1
death                     1
other_aid                 1
infrastructure_related    1
transport                 1
buildings                 1
electricity               1
tools                     1
hospitals                 1
shops                     1
aid_centers               1
other_infrastructure      1
weather_related           1
floods                    1
storm                     1
fire                      1
earthquake                1
cold                      1
other_weather             1
direct_report       

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    words = word_tokenize(re.sub(r"\W", " ", text.lower()))
    tokens = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize, token_pattern=None)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(token_pattern=None,
                                 tokenizer=<function tokenize at 0x7f9fa5fa1b40>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = pipeline.predict(X_test)

for index, column in enumerate(y_test):
    print(f"{column} accuracy: {accuracy_score(y_test[column], y_pred[:,index])}\n{classification_report(y_test[column], y_pred[:, index])}")

related accuracy: 0.8235742895288957
              precision    recall  f1-score   support

           0       0.70      0.40      0.51      1199
           1       0.84      0.95      0.89      4044

    accuracy                           0.82      5243
   macro avg       0.77      0.67      0.70      5243
weighted avg       0.81      0.82      0.80      5243

request accuracy: 0.8884226587831394
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4314
           1       0.81      0.48      0.60       929

    accuracy                           0.89      5243
   macro avg       0.86      0.73      0.77      5243
weighted avg       0.88      0.89      0.88      5243



/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

offer accuracy: 0.9958039290482548
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5221
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00      0.99      5243

aid_related accuracy: 0.7818043105092505
              precision    recall  f1-score   support

           0       0.80      0.84      0.82      3098
           1       0.75      0.70      0.73      2145

    accuracy                           0.78      5243
   macro avg       0.78      0.77      0.77      5243
weighted avg       0.78      0.78      0.78      5243

medical_help accuracy: 0.9212283044058746
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4816
           1       0.64      0.07      0.13       427

    accuracy                           0.92      5243
   macro a

/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

hospitals accuracy: 0.9891283616250238
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5186
           1       0.00      0.00      0.00        57

    accuracy                           0.99      5243
   macro avg       0.49      0.50      0.50      5243
weighted avg       0.98      0.99      0.98      5243

shops accuracy: 0.996185390043868
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5223
           1       0.00      0.00      0.00        20

    accuracy                           1.00      5243
   macro avg       0.50      0.50      0.50      5243
weighted avg       0.99      1.00      0.99      5243



/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


aid_centers accuracy: 0.9877932481403776
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5179
           1       0.00      0.00      0.00        64

    accuracy                           0.99      5243
   macro avg       0.49      0.50      0.50      5243
weighted avg       0.98      0.99      0.98      5243

other_infrastructure accuracy: 0.9597558649628075
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5034
           1       0.00      0.00      0.00       209

    accuracy                           0.96      5243
   macro avg       0.48      0.50      0.49      5243
weighted avg       0.92      0.96      0.94      5243

weather_related accuracy: 0.8832729353423613
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      3805
           1       0.85      0.70      0.77      1438

    accuracy                           0.88   

/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nhanvo/anaconda3/envs/python-lab/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

earthquake accuracy: 0.9700553118443639
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4749
           1       0.89      0.78      0.83       494

    accuracy                           0.97      5243
   macro avg       0.93      0.89      0.91      5243
weighted avg       0.97      0.97      0.97      5243

cold accuracy: 0.9811176807171467
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5139
           1       0.67      0.10      0.17       104

    accuracy                           0.98      5243
   macro avg       0.82      0.55      0.58      5243
weighted avg       0.98      0.98      0.97      5243

other_weather accuracy: 0.9513637230593172
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4988
           1       0.50      0.03      0.05       255

    accuracy                           0.95      5243
   macro av

In [7]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect', CountVectorizer(token_pattern=None,
                   tokenizer=<function tokenize at 0x7f9fa5fa1b40>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(token_pattern=None,
                 tokenizer=<function tokenize at 0x7f9fa5fa1b40>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': None,
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'clf__estimator__n_estimators': [100, 500],
    'clf__estimator__min_samples_split': [2, 4, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=4)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.best_params_

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.